ＴＯＤＯ：簡介

draw.io 圖片

## 概念

In [ ]:
workflow management system (WMS)
Workflow, DAG, operator, task, task instance, operator
Sensor, operator
meta-data db, scheduler, webserver, log, ui

In [ ]:
#blockquote
The Airflow Platform is a tool for describing, executing, and monitoring workflows.

![](images/airflow/airflow-architecture.png)

小知識
- 預設用 sqlite

## 安裝

In [ ]:
conda create -n airflow python=3 -y

source activate airflow

export AIRFLOW_HOME="$(pwd)"

echo $AIRFLOW_HOM

pip install apache-airflow

pip install cryptography

In [ ]:
python -c "from cryptography.fernet import Fernet; print(Fernet.generate_key().decode())"

## local test

In [6]:
from airflow import DAG

In [5]:
from airflow.operators.bash_operator import BashOperator

In [7]:
from datetime import datetime, timedelta

In [9]:
from airflow.operators.python_operator import PythonOperator

In [10]:
PythonOperator?

## 常用 CLI 指令

### 查看類

看有什麼 DAGs
```bash
airflow list_dags
```

看某個 DAG 裡頭有什麼 tasks （以樹狀呈現）。要注意呈現的樹狀從左到右， dependencies 是從下到上。
```bash
airflow list_tasks DAG_ID --tree

<Task(BashOperator): transform>
    <Task(BashOperator): extract>
```

### 測試 tasks
測試的特性：
- 不管該 task 的 dependencies，直接執行該 task。
- 不會把 log 存起來，而是直接丟到 stdout 讓我們查看。

基本格式：
```bash
airflow test DAG_ID TASK_ID EXECUTE_DATE
```

範例：
```bash
airflow test etl extract 2018-03-16
```

### Backfill
主要應用應該是在建立 / 測試好一個新的 DAG 以後，把過去的資料也同樣做轉換。使用 [Datestamp 做 Data Partition 可以讓 Data Backfilling 更有效率](https://towardsdatascience.com/a-beginners-guide-to-data-engineering-part-ii-47c4e7cbda71)。這樣我們就可以只使用一個 ETL flow 就把新舊資料做完轉換。

特性
- 會看 tasks 之間的 dependencies。
- 這邊的時間是模擬此 DAG 在這個時間範圍內執行。
- 結果會輸出到硬碟上的 log 檔案以及資料庫裡頭。

基本格式：
```bash
airflow backfill DAG_ID -s START_DATE -e END_DATE
```

範例：
```bash
airflow backfill etl -s 2018-03-05 -e 2018-03-06
```


### 雜項
刪除某 DAG 所有的 task instances
```bash
airflow clear DAG_ID
```

清空 db
```bash
airflow resetdb
```

讓 scheduler 開始監控 DAG
```bash
airflow unpause DAG_ID
```


## 推薦閱讀
- Airflow 作者的 [The Rise of Data Engineer](https://medium.freecodecamp.org/the-rise-of-the-data-engineer-91be18f1e603)
- Airbnb 的資料科學家 Robert Chang 對 [Data Enginner 的介紹]((https://medium.com/@rchang/a-beginners-guide-to-data-engineering-part-i-4227c5c457d7).
    * data warehousing(DWH) 的重要以及與其息息相關的 ETL 的關係
    * ETL (Extract, Transform, Load）是如何對應到 Airflow 的 Sensor, Operator
    * JVM-centric ETL 跟 SQL-centric ETL 的比較
- [Quizlet 介紹他們將 Airflow 當作 WMS 使用的系列文](https://medium.com/tech-quizlet/going-with-the-flow-how-quizlet-uses-apache-airflow-to-execute-complex-data-processing-pipelines-1ca546f8cc68)
    * [Part1: 為何要使用CRON以外的方法？](https://medium.com/@dustinstansbury/beyond-cron-an-introduction-to-workflow-management-systems-19987afcdb5e)
        * WMS 比 CRON 好的地方
            * handle task dependencies
            * error handling, retry the task and all dependencies
            * 專案越多使用 CRON 就有用多種用法，沒有 owners 難以管理
    * [Part2：一個好的WMS要有什麼功能？](https://towardsdatascience.com/why-quizlet-chose-apache-airflow-for-executing-data-workflows-3f97d40e9571)
        * smart scheduling, 定期要跑的 task 超過他預期跑的時間怎解？
        * 依賴管理，失敗的 task 以及相關的 tasks 要能 retry, 沒有依賴的 task 要能最有效率先跑
        * 當 task 失敗或者是違反 SLA 時通知我們
        * pipeline 是用程式定義的，而非寫死 (e.g., Airflow 的 Python, Jinja)
        * 其他：scalability, flexibility(什麼 task 都能處理)
    * [Part3：實際Airflow的操作](https://medium.com/@dustinstansbury/understanding-apache-airflows-key-concepts-a96efed52b1a)
        * Define workflow in code 的好處
        * 如何透過繼承 BaseOperator 自訂 Sensor, Operator
        * Airflow 結構, 各個 components 在做什麼 (scheduler, webserver, executers, UI, meta-data DB)
    * [Part4: 實際 deploy 的注意事項以及 Airflow 缺點](https://medium.com/@dustinstansbury/how-quizlet-uses-apache-airflow-in-practice-a903cbb5626d)
        * Airflow 不適合 streaming data, 如果需要可以考慮 [Google Dataflow](https://cloud.google.com/dataflow/)
        * 還有一些小細節值得注意，像是 `skipped != success`, 不要隨便換 `DAG_ID`
- [Airflow 的注意事項(Gotcha)](https://gtoonstra.github.io/etl-with-airflow/gotchas.html)
    
    
* import python scripts in subdirectory
    * https://stackoverflow.com/questions/1260792/import-a-file-from-a-subdirectory
* 跑 python script
    * https://stackoverflow.com/questions/39610313/how-to-use-airflow-to-run-a-list-of-python-tasks

出現找不到 dag 然後確定有用 python 通過 parse 測驗以後, 確認目前的 airflow 的執行路徑, 不對的話就設定 export ...

## 疑難雜症
- 產生 fernet key: https://bcb.github.io/airflow/fernet-key
- [task instance table not found](https://groups.google.com/forum/#!searchin/airbnb_airflow/upgradedb%7Csort:date/airbnb_airflow/ueaGk6FFtqw/KGpmMuzVDgAJ)
- UI break: https://github.com/puckel/docker-airflow/issues/116

In [ ]:
#ignore_from_here

## ME 應用
將公司所有 ETL / data workflow 整合管理. fault-torelance (retry) 確保

### TODO
- 與 AWS lambda 的整合, 看 airflow repo



### 實例
- series, maker master 等爬下來的資料，如果之後想用別的 logic 重新 parse, 可以使用 backfilling
- mongo2 -> es, 失敗的時候不止送email 可以做別的處理
- 孫現在做的 biztel 也一樣
- NPS 定期更新
- 未來 marketing, ReRe 的 google data + sales data + google data studio



